In [1]:
%run alpha.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [2]:
# kinematics and dynamics code of these casadi objects can be found in https://github.com/edxmorgan/diff_uv project
# these can be replaced with your preferred uv dynamics
fb_id_func = cs.Function.load("float_base.casadi")
J_uv_func = cs.Function.load("J_uv.casadi")


# ned jacobian for uv kinematics
J_uv = J_uv_func(ss.eul)

# base bias force expressions
B_F = fb_id_func(ss.v_base, ss.v_c, ss.eul, [0,0,0,0,0,0]) 

# base inverse dynamics expressions
fb_tau = fb_id_func(ss.v_base, ss.v_c, ss.eul, ss.a_base)

# uvms bias force and Inertia
C = rig_dyn.get_bias_force(gravity=9.81, floating_base_bias_f=B_F)
H = rig_dyn.get_inertia_matrix(gravity=9.81, floating_base_id=fb_tau, floating_base_bias_f=B_F)

floating_base found
floating_base found
floating_base found


In [3]:
H_eval = cs.Function(
    'H', [ss.q, ss.q_dot, ss.eul, ss.v_base, ss.uv_u, ss.rigid_body_p, ss.hydrodynamic_p, ss.v_c, ss.base_T, ss.trivial_Ir], [H])
C_eval = cs.Function(
    'C', [ss.q, ss.q_dot, ss.eul, ss.v_base, ss.uv_u, ss.rigid_body_p, ss.hydrodynamic_p, ss.v_c, ss.base_T, ss.trivial_Ir], [C])

In [ ]:
qq, qqd = [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]
base_T0 = [3.142, 0.000, 0.000, 0.140, 0.000, -0.120] #transformation of uv body frame to manipulator base
v0 = [0, 0, 0, 0, 0, 0]
eul0 = [0, 0, 0]
uv_u = [0, 0, 0, 0, 0, 0]

H_eval(qq, qqd, eul0 , v0, uv_u, rigid_body_p_est, arm_Hyd__p_est, v_c0, base_T0, trivial_Ir0)

DM(
[[0.262535, -0.0042473, -0.0121885, -0.131521, -0.277469, 0.00380784, 0.00555238, 0.0257429, -0.00685153, 0.000187603], 
 [0.00464013, 0.296479, -0.00730201, 0.262544, -0.131659, -0.21781, -0.0111173, -0.0168617, 0.0171562, -0.000102593], 
 [0.0232646, 0.000331969, 0.305361, 0.0894041, 0.120606, -0.000137874, -0.0310174, -0.0152534, 0.00223177, 0.00334353], 
 [0.00178113, 0.117986, -0.026767, 17.5395, -0.484105, 0.000197198, -0.0409945, 0.0262621, -0.00732366, -0.000490842], 
 [-0.0875932, 0.00190335, 0.130703, 0.484105, 24.9085, -0.000833631, -0.0315439, -0.082628, 0.0122829, -0.000870158], 
 [-0.00211012, -0.169333, 0.000122219, 0.000197198, 0.000833631, 27.408, -1.28493e-05, 0.00355003, 0.00809984, -3.54456e-07], 
 [-1.47584e-06, -3.06556e-08, -3.13552e-06, -9.59074e-06, -9.52879e-06, 3.88152e-09, 0.0225372, 1.63254e-06, -2.23984e-07, -1.53776e-06], 
 [-1.87205e-06, -3.19164e-06, 2.03065e-06, 1.16537e-05, 3.66747e-05, 2.31102e-05, 3.10308e-06, 0.0225339, -8.37271e-06, 2.2922e-08

In [5]:
# for H's decoupled version, compare to these matrix
#[Hv 0]
#[0 Hm]

# vehicle base inertia matrix, Hv :
# [[ 0.28  0.    0.    0.   -0.23  0.  ]
#  [ 0.    0.28  0.    0.23  0.    0.  ]
#  [ 0.    0.    0.28  0.    0.    0.  ]
#  [ 0.    0.23  0.   17.    0.    0.  ]
#  [-0.23  0.    0.    0.   24.2   0.  ]
#  [ 0.    0.    0.    0.    0.   26.07]]

# Manipulator inertia matrix, Hm :
# M = rig_dyn.get_inertia_matrix()
# M_eval = cs.Function('Mr', [ss.q, ss.q_dot, ss.rigid_body_p, ss.hydrodynamic_p, ss.base_T], [M])
# M_eval(qq, qqd, rigid_body_p_est, arm_Hyd__p_est, base_T0)


In [6]:
C_eval(qq, qqd, eul0 , v0, uv_u, rigid_body_p_est, arm_Hyd__p_est, v_c0, base_T0, trivial_Ir0)

DM([0.183459, -3.68097, 0, 6.03767e-20, 0, 12.4876, -3.99586e-07, 0.000816605, -0.000153325, 3.51455e-07])

In [7]:
# for B_F's decoupled version, compare to these matrix
#[B_Fv]
#[C_Fm]

# B_F_func = cs.Function('B_F', [ss.v_base, ss.v_c, ss.eul], [B_F]) # for numerical & symbolic use
# B_Fv = B_F_func([0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0])